In [ ]:
!pip install m3gnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.0/809.0 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 58.8 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow==2.13.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 41.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.1
    Uninstalling typing_extensions-4.13.1:
      Successfully uninstalled typing_extensions-4.13.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting u

In [ ]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spglib 2.6.0 requires typing-extensions>=4.9.0; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
tensorflow 2.13.1 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.4 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.13.1 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.13.1 which is incompatible.
nibabel 5.3.2 requires typing-extensions>=4.6; python_version < "3.13

In [ ]:
# Mount Google Drive in Colab
#!pip install matgl
from google.colab import drive
drive.mount('/content/drive')

import os
import csv
from ase.constraints import FixAtoms
from ase.io import read, write
from ase.mep import NEB
from ase.optimize import BFGS, QuasiNewton
from m3gnet.models import M3GNet, Potential, M3GNetCalculator, Relaxer
# Set device to GPU (A100)
device = 'gpu'  # or 'cuda:0' if preferred
base_input_dir = '/content/drive/MyDrive/MLIP_NEB_inputs'

# Walk through the directory tree under MLIP_NEB_inputs
for root, dirs, files in os.walk(base_input_dir):
    # Check if both "initial" and "final" folders are present in the current directory
    if 'initial' in dirs and 'final' in dirs:
        initial_path = os.path.join(root, "initial", "POSCAR")
        final_path = os.path.join(root, "final", "POSCAR")

        # Only process if both POSCAR files exist
        if not (os.path.exists(initial_path) and os.path.exists(final_path)):
            print(f"Skipping {root}: initial or final POSCAR not found.")
            continue

        system_folder = root  # This is the system's folder
        system_name = os.path.basename(system_folder)
        print(f"Processing system: {system_name}")

        # Create a subfolder in the system folder for the NEB outputs
        save_dir = os.path.join(system_folder, "M3GNet_RUN")
        os.makedirs(save_dir, exist_ok=True)

        # CSV file to record energies
        csv_path = os.path.join(save_dir, "energies.csv")

        # Read initial and final structures
        initial = read(initial_path)
        final = read(final_path)

        # Apply constraints: fix atoms with tag > 1
        mask_initial = [atom.tag > 1 for atom in initial]
        initial.set_constraint(FixAtoms(mask=mask_initial))
        mask_final = [atom.tag > 1 for atom in final]
        final.set_constraint(FixAtoms(mask=mask_final))
        # We'll use the final mask for the NEB images

        # Set up M3GNet calculator and assign it to both endpoints
        potential = Potential(M3GNet.load())
        calc = M3GNetCalculator(potential=potential)
        initial.calc = calc
        final.calc = calc

        # Relax the initial structure
        init_traj = os.path.join(save_dir, "initial_relaxed.traj")
        qn = QuasiNewton(initial, trajectory=init_traj)
        qn.run(fmax=0.05, steps=1000)
        write(os.path.join(save_dir, "relaxed_initial.vasp"), initial, format="vasp")

        # Relax the final structure
        final_traj = os.path.join(save_dir, "final_relaxed.traj")
        qn = QuasiNewton(final, trajectory=final_traj)
        qn.run(fmax=0.05, steps=1000)
        write(os.path.join(save_dir, "relaxed_final.vasp"), final, format="vasp")

        # Reload relaxed structures
        initial = read(init_traj)
        final = read(final_traj)

        # Create NEB images (7 intermediate images)
        constraint = FixAtoms(mask=mask_final)
        images = [initial]
        for i in range(7):
            image = initial.copy()
            # Assign a fresh CHGNet calculator to each image
            potential = Potential(M3GNet.load())
            calc = M3GNetCalculator(potential=potential)
            image.set_constraint(constraint)
            images.append(image)
        images.append(final)

        # Set up and run the NEB calculation
        neb = NEB(images, parallel=True, k=5, method='eb')
        neb.interpolate('idpp')
        neb_traj = os.path.join(save_dir, "neb.traj")
        qn = BFGS(neb, trajectory=neb_traj)
        qn.run(fmax=0.05, steps=1000)

        # Compute and save energies for each image
        energies = []
        with open(csv_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Image Index", "Energy (eV)"])
            for i, image in enumerate(images):
                try:
                    energy = image.get_potential_energy()
                except Exception as e:
                    print(f"Error computing energy for image {i} of {system_name}: {e}")
                    energy = None
                energies.append(energy)
                output_file = os.path.join(save_dir, f"neb_optimized_{i}.vasp")
                write(output_file, image, format="vasp")
                writer.writerow([i, energy])

        # Check for any energy errors before calculating migration barrier
        if None in energies:
            print(f"Skipping migration barrier calculation for {system_name} due to energy errors")
            continue

        migration_barrier = max(energies) - min(energies)
        print(f"{system_name} - Migration Barrier: {migration_barrier:.6f} eV")
        with open(csv_path, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([])
            writer.writerow(["Migration Barrier", migration_barrier])

print("All NEB Calculations complete!")


